In [19]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tabulate import tabulate
import plotly.express as px

### Variables & Parameters

In [20]:
from ortools.linear_solver import pywraplp

# Create the solver with the GLOP solver
solver = pywraplp.Solver.CreateSolver('SCIP')

In [21]:
# Single value parameters
pmax = 500  # m3/s
qmax = 200  # m3/s
vmax = 200  # hm3
v0 = 80  # hm3
C = 0.0864  # m3/s to hm3/day
days = 10  # number of time steps
inflow_scaling_factor = 25  # inflow scaling factor to create interesting model

# Solar panel parameters
solar_constant = 1.37 #kW/m2
sunlight_hours = 8 #hours
solar_to_elect = 0.20 # kW of sun to kW of electricity
sunlight_constant = solar_constant * sunlight_hours # kWh/m2/day
solar_panel_area = 12 #m2 of solar panels PARAMETER

# Vector parameters
rev = np.array([1.00, 0.05, 0.1, 0.1, 0.1, 0.25, 0.80, 2.00, 2.00, 1.00])  # created, not from height, revenue per unit of energy (1 being baseline (100%) normal price, 2 being double price, 0.5 being half price)
## Some days, it is more profitable to store water than to sell it
a = np.array([1, 2, 16, 16, 8, 2, 8, 4, 16, 1]) * inflow_scaling_factor
sun_pred = np.array([1, 0.8, 1.2, 1.5, 0.1, 0.2, 0.3, 1, 0.9, 0.8]) # predicted sunlight for each day as percentage of nominal sunlight (solar_constant)
#TODO CONVERT TO ACTUAL SUNLIGHT FOR SOLAR_CONSTANT

# Empty array to fill in with pumped water
#QUESTION: Should this be an implicit variable that gets filled in by the solver?
pumped_water = np.array([])

# External variables
p = {}  # p[t] is the flow through the turbines at time t
q = {}  # q[t] is the flow dumped in the lake at time t

# Implicit variables
v = {}  # v[t] is the volume in reservoir at time t, hm3
prod = {}  # prod[t] is the energy production * revenue per energy unit

In [22]:
# I think everything could be in percentage of grid capacity to simplify and to make it more general
## FOR EXAMPLE: there are solar panels to produce 40% of energy needed in the area NOMINALY, on a 1x sunny day, 40% of energy is solar. On a 2x sunny day, 80% of energy is solar etc.
## Same for wind.
# Number of solar panels in the area

# Maximum capacity of the grid OF ALPINE AREA that feeds a small village and is linked with hydro
max_grid = 250 #kWh
# QUESTION: This

# Conversion of electricity to water
elec_to_water = 0.60

In [23]:
# Create variables, N+1 to accommodate for v[t+1] term
for t in range(days+1):
    p[t] = solver.NumVar(0, solver.infinity(), 'p[%i]' % t)
    q[t] = solver.NumVar(0, solver.infinity(), 'q[%i]' % t)
    v[t] = solver.NumVar(0, solver.infinity(), 'v[%i]' % t)
    prod[t] = solver.NumVar(0, solver.infinity(), 'prod[%i]' % t)

print('Number of variables =', solver.NumVariables())

Number of variables = 44


In [24]:
for t in range(days):
    # Initial conditions
    solver.Add(v[0] == v0)  # initial volume
    
    # Calculate electricity produced from solar panels
    sol_pow = sunlight_constant * sun_pred[t] * solar_panel_area * solar_to_elect
    # kWh/m2/day * unitless * m2 * kWh/m2 = kWh/day
    print("Day", t, "Solar power:", round(sol_pow,2))
    # Check if the electricity produced is higher than the maximum supported from the grid
    if sol_pow > max_grid:
        # If it is higher, calculate the surplus
        elec_surplus = sol_pow - max_grid    
    else:
        # # If it is lower, set the surplus to zero
        elec_surplus = 0
   
    # Calculate the water that could be pumped into the dam (Converting kWh/day to hm3/day) https://www.quora.com/How-many-kW-is-needed-to-pump-1000-m-3-h-of-water-to-10-m-high
    ## Volume pumped from energy amount
    ## We need to convert kWh/day surplus to hm3/day added water
    additional_water  = elec_surplus * elec_to_water

    # Append the value in a numpy array
    pumped_water = np.append(pumped_water, np.int32(additional_water))

    # Constraints that contain [t-1] terms
    solver.Add(v[t+1] == v[t] + pumped_water[t] + C * (a[t] - p[t] - q[t]))  # volume balance
    solver.Add(p[t] - p[t+1] <= 0.5 * pmax)
    solver.Add(p[t+1] - p[t] <= 0.5 * pmax)

    # Production from efficiency and flow in turbine
    solver.Add(prod[t] == rev[t] * p[t])

    # Ceiling constraints
    solver.Add(v[t] <= vmax)
    solver.Add(p[t] <= pmax)
    solver.Add(q[t] <= qmax)

    # Non-negativity constraints
    solver.Add(v[t] >= 0)
    solver.Add(p[t] >= 0)
    solver.Add(q[t] >= 0)
    solver.Add(prod[t] >= 0)


Day 0 Solar power: 26.3
Day 1 Solar power: 21.04
Day 2 Solar power: 31.56
Day 3 Solar power: 39.46
Day 4 Solar power: 2.63
Day 5 Solar power: 5.26
Day 6 Solar power: 7.89
Day 7 Solar power: 26.3
Day 8 Solar power: 23.67
Day 9 Solar power: 21.04


In [25]:
# Objective function
objective = solver.Objective()

# Equilvalent to sum(prod[t] for t in range(N)) or prod[0] + ... + prod[days - 1]
for t in range(days):
    objective.SetCoefficient(prod[t], 1)
objective.SetMaximization()

# Solve
status = solver.Solve()

In [26]:
# Extract solution
p_sol = []
q_sol = []
v_sol = []
prod_sol = []
print('Objective value =', solver.Objective().Value())
# If optimality is reached, print the solution
if status == pywraplp.Solver.OPTIMAL:
    print('Objective value =', solver.Objective().Value())
    for t in range(days + 1):
        print(p[t].name(), ' = ', p[t].solution_value())
        print(q[t].name(), ' = ', q[t].solution_value())
        print(v[t].name(), ' = ', v[t].solution_value())
        p_sol.append(p[t].solution_value())
        q_sol.append(q[t].solution_value())
        v_sol.append(v[t].solution_value())
        prod_sol.append(prod[t].solution_value())
        print()

Objective value = 3357.361111111111
Objective value = 3357.361111111111
p[0]  =  387.9629629629624
q[0]  =  1.1368683772161603e-13
v[0]  =  80.0

p[1]  =  137.9629629629624
q[1]  =  1.7053025658242404e-13
v[1]  =  48.64000000000004

p[2]  =  0.0
q[2]  =  1.1368683772161603e-13
v[2]  =  41.04000000000008

p[3]  =  0.0
q[3]  =  2.2737367544323206e-13
v[3]  =  75.60000000000008

p[4]  =  0.0
q[4]  =  0.0
v[4]  =  110.16000000000008

p[5]  =  250.00000000000006
q[5]  =  4.547473508864641e-13
v[5]  =  127.44000000000007

p[6]  =  500.00000000000006
q[6]  =  1.1368683772161603e-13
v[6]  =  110.16000000000003

p[7]  =  500.0000000000001
q[7]  =  0.0
v[7]  =  84.24000000000002

p[8]  =  500.0000000000001
q[8]  =  -1.1368683772161603e-13
v[8]  =  49.68000000000001

p[9]  =  500.0000000000001
q[9]  =  0.0
v[9]  =  41.040000000000006

p[10]  =  750.0000000000001
q[10]  =  -0.0
v[10]  =  -7.105427357601002e-15



In [27]:
df = pd.DataFrame(data = [p_sol, q_sol, a, v_sol, prod_sol, rev, pumped_water]).T
df.columns = ['p_t', 'q_t', 'a_t', 'v_t', 'prod_t', 'eff', "pumped_water"]
df = df.round(2)
#df.style.format(precision = 2)
print('Volume at the end of last day =', v_sol[-1])
df=df[:-1]
print(tabulate(df, headers = df.columns))
# graph all but last row on plotly
fig = px.line(df, x = df.index, y = df.columns)
fig.show()

Volume at the end of last day = -7.105427357601002e-15
       p_t    q_t    a_t     v_t    prod_t    eff    pumped_water
--  ------  -----  -----  ------  --------  -----  --------------
 0  387.96      0     25   80       387.96   1                  0
 1  137.96      0     50   48.64      6.9    0.05               0
 2    0         0    400   41.04      0      0.1                0
 3    0         0    400   75.6       0      0.1                0
 4    0         0    200  110.16      0      0.1                0
 5  250         0     50  127.44     62.5    0.25               0
 6  500         0    200  110.16    400      0.8                0
 7  500         0    100   84.24   1000      2                  0
 8  500        -0    400   49.68   1000      2                  0
 9  500         0     25   41.04    500      1                  0
